# ECE 285 – MLIP – Assignment 1


## 1

In [ ]:
import MNISTtools
xtrain, ltrain = MNISTtools.load(dataset="training", path="/datasets/MNIST/")
print(xtrain.shape)
print(ltrain.shape)
print('done 1')

1.What are the shapes of both variables? 
<br>ans : (784, 60000) and (60000,1)
<br>2.What is the size of the training dataset? 
<br>ans : 60000
<br>3.What is the feature dimension? 
<br>ans : 784

## 2 

In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
w, h = 28, 28
img_42 = xtrain[:,42]
img_42 = np.reshape(img_42, (28, 28))
img = Image.fromarray(img_42, 'L')
plt.imshow(img, cmap = "gray")
print(ltrain[42])
print('done 2')

7
done 2


check that its content corresponds to its label: checked

## 3

In [3]:
import numpy as np
minElement = np.amin(xtrain)
maxElement = np.amax(xtrain)
print(minElement)
print(maxElement)
print(type(xtrain))
print('done 3')

0
255
<class 'numpy.ndarray'>
done 3


1.What is the range of xtrain (minimum and maximum values)? 
<br>ans : 0,255
<br>2.What is the type of xtrain? 
<br>ans : <class 'numpy.ndarray'>

## 4

In [4]:
def normalize_MNIST_images(x):
    y = x
    y.astype(np.float32)
    y = y*2/255 -1
    return y
xtrain_normalized = normalize_MNIST_images(xtrain)
print('done 4')

done 4


## 5

In [5]:
def label2onehot(lbl):
    d = np.zeros((lbl.max() + 1, lbl.size))
    d[lbl, np.arange(lbl.size)] = 1
    return d
dtrain = label2onehot(ltrain)
print(dtrain[:,42])
print(ltrain[42])
print('done 5')

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
7
done 5


Make sure that the one-hot code dtrain[:,42]
corresponds to ltrain[42] : checked

## 6

In [6]:
def onehot2label(d):
    lbl = np.argmax(d,axis=0)
    return lbl
print('done 6')

done 6


## 7

In [7]:
def softmax(a):
    a.astype(np.float32)
    b = np.amax(a, axis=0)
    b = np.reshape(b,(1,b.shape[0]))
    b = np.tile(b,(a.shape[0],1))
    c = a - b
    exp_a = np.exp(c)
    d = np.sum(exp_a,axis=0)
    d = np.reshape(d,(1,d.shape[0]))
    d = np.tile(d,(exp_a.shape[0],1))   
    y = np.divide(exp_a,d)
    return y
print('done 7')

done 7


## 8

We have $g(a)_i = \frac{exp(a_i)}{\sum_{j=1}^{N} exp(a_j)}$ ... (1)
<br>and $\frac{d}{dx} \frac{f}{g} = \frac{f'g-fg'}{g^2}$ ... (2)
<br>Plug (1) into (2) we obtain 
$\frac{\partial g(a)_i}{\partial a_i} = $ 
$\frac{exp(a_i)\times \sum_{j=1}^{N} exp(a_j)-exp(a_i)\times exp(a_i)}{(\sum_{j=1}^{N} exp(a_j))^2} = $
$\frac{exp(a_i)\times(\sum_{j=1}^{N} exp(a_j)-exp(a_i))}{(\sum_{j=1}^{N} exp(a_j))^2} $
<br>Which equals to 
$\frac{exp(a_i)}{\sum_{j=1}^{N} exp(a_j)}$
$\times \frac{(\sum_{j=1}^{N} exp(a_j)-exp(a_i))}{\sum_{j=1}^{N} exp(a_j)} = $
$g(a)_i\times (1-g(a)_i)$

# 9

We have $g(a)_i = \frac{exp(a_i)}{\sum_{n=1}^{N} exp(a_n)}$ ... (1)
<br>and $\frac{d}{dx} \frac{f}{g} = \frac{f'g-fg'}{g^2}$ ... (2)
<br>Plug (1) into (2) we obtain 
$\frac{\partial g(a)_i}{\partial a_j} = $ 
$\frac{0-exp(a_i)\times exp(a_j)}{(\sum_{n=1}^{N} exp(a_n))^2} = $
$-\frac{exp(a_i)}{\sum_{n=1}^{N} exp(a_n)}\times \frac{exp(a_j)}{\sum_{n=1}^{N} exp(a_n)}$
<br>Which equals to 
$-g(a)_i\times g(a)_j$

## 10

$\delta = [\frac{\partial g(a)}{\partial a}]^T \times e$ ...(1)
<br>Where $[\frac{\partial g(a)}{\partial a}]_{ii} = g(a)_i(1-g(a)_i)$ and $[\frac{\partial g(a)}{\partial a}]_{ij} = -g(a)_ig(a)_j$ if $i\ne j$ 
<br>So $[\frac{\partial g(a)}{\partial a}]_{ij} =[\frac{\partial g(a)}{\partial a}]_{ji}$  and thus the Jacobian of softmax is symmetric.
<br>Therefore $[\frac{\partial g(a)}{\partial a}]^T = [\frac{\partial g(a)}{\partial a}] = A-B$ 
<br>Where $A_{ii} = g(a)_i$ and $A_{ij} = 0$ if $i\ne j$, 
<br>and $B_{ij} = g(a)_ig(a)_j$
<br>Plugging into (1) we have $\delta = (A-B)\times e = A\times e-B\times e$ 
<br>So it is obvious that $A\times e =  g(a)\bigotimes e$
<br>and $B\times e$ is a column vector with the element of row i being $(\sum_{j=1}^{N} g(a)_je_j)g(a)_i$, and this is exactly $\langle g(a),{e} \rangle g(a)$
<br>So eventually we have $\delta = g(a)\bigotimes e - \langle g(a),{e} \rangle g(a)$

In [8]:
def softmaxp(a, e):
    delta = np.zeros((a.shape[0],a.shape[1]))
    c = softmax(a)
    for i in range(a.shape[1]):
        delta[:,i] = np.multiply(c[:,i],e[:,i])
    b = np.zeros((a.shape[0],a.shape[1]))
    for i in range(a.shape[1]): 
        b[:,i] = np.matmul(np.transpose(c[:,i]),e[:,i])*c[:,i]
    delta = delta - b
    return delta
print('done 10')

done 10


# 11

In [9]:
import numpy as np
eps = 1e-6 # finite difference step
a = np.random.randn(10, 200) # random inputs
e = np.random.randn(10, 200) # random directions
## treat a, e as 200 column vectors (i.e. batch size = 200)
diff = softmaxp(a, e)
diff_approx = (softmax(a+eps*e)-softmax(a))/eps
rel_error = np.abs(diff - diff_approx).mean() / np.abs(diff_approx).mean()
print(rel_error, 'should be smaller than 1e-6')
print('done 11')

5.039340807622801e-07 should be smaller than 1e-6
done 11


4.82515827998792e-07 is smaller than 1e-6 !!

# 12 

In [10]:
def relu(a):
    zeros = np.zeros(a.shape)
    relu = np.maximum(a,zeros)
    return relu
def relup(a, e):
    relu_p = np.zeros(e.shape)
    for i in range (e.shape[1]): 
        j_relu = np.zeros((a.shape[0],a.shape[0]))
        for j in range(a.shape[0]):
            if a[j,i]>0:
                j_relu[j][j] = 1
        relu_p[:,i] = np.matmul(j_relu,e[:,i])
    return relu_p
eps = 1e-6 # finite difference step
a = np.random.randn(10, 200) # random inputs
e = np.random.randn(10, 200) # random directions
diff = relup(a, e)
diff_approx = (relu(a+eps*e)-relu(a))/eps
rel_error = np.abs(diff - diff_approx).mean() / np.abs(diff_approx).mean()
print(rel_error, 'should be smaller than 1e-6')
print('done 12')

3.9586013049732394e-11 should be smaller than 1e-6
done 12


The jacobian matrix of relu function = J
$J_{ii} = 1$ if $a_i>0$ and $J_{ii} = 0$ if $a_i<=0$ 
<br>(not differentialble at 0 lol)
<br> and $J_{ij} = 0$ if $i\ne j$
<br> 3.9505616765734874e-11 is smaller than 1e-6 !!


# 13

In [11]:
def init_shallow(Ni, Nh, No):
    b1 = np.random.randn(Nh, 1) / np.sqrt((Ni+1.)/2.)
    W1 = np.random.randn(Nh, Ni) / np.sqrt((Ni+1.)/2.)
    b2 = np.random.randn(No, 1) / np.sqrt((Nh+1.))
    W2 = np.random.randn(No, Nh) / np.sqrt((Nh+1.))
    return W1, b1, W2, b2

Ni = xtrain.shape[0]
Nh = 64
No = dtrain.shape[0]
netinit = init_shallow(Ni, Nh, No)
print('done 13')

done 13


The method of dividing np.sqrt((Ni+1.)/2. and np.sqrt((Nh+1.)) are called He and Xavier initializations！！

# 14

In [12]:
def forwardprop_shallow(x, net):
    W1 = net[0]
    b1 = net[1]
    W2 = net[2]
    b2 = net[3]
    a1 = W1.dot(x) + b1
    h1 = relu(a1)
    a2 = W2.dot(h1) + b2
    y = softmax(a2)
    return y
yinit = forwardprop_shallow(xtrain_normalized, netinit)
print('done 14')

done 14


# 15

In [13]:
def eval_loss(y, d):
    loss = np.zeros((1,1))
    loss.astype(float)
    print('y.shape = ',y.shape)
    print('d.shape = ',d.shape)
    for i in range(d.shape[1]):
        p = np.dot(d[:,i],np.log(y[:,i]))
        p = p/d.shape[1]
        p = p*(-1)
        if np.isnan(p):
            loss = loss
        elif np.isinf(p):
            loss = loss
        else :
            loss = loss + p
        if i%2000==0:
            print('loss = ',loss)
    return loss
print(eval_loss(yinit, dtrain), 'should be around .26')
print('done 15')

y.shape =  (10, 60000)
d.shape =  (10, 60000)
loss =  [[6.49717818e-05]]
loss =  [[0.09196369]]
loss =  [[0.18338713]]
loss =  [[0.27530654]]
loss =  [[0.36836786]]
loss =  [[0.45980052]]
loss =  [[0.55210518]]
loss =  [[0.64359702]]
loss =  [[0.73596353]]
loss =  [[0.82734432]]
loss =  [[0.92058244]]
loss =  [[1.01236992]]
loss =  [[1.10496438]]
loss =  [[1.19713488]]
loss =  [[1.28958501]]
loss =  [[1.38277178]]
loss =  [[1.47504083]]
loss =  [[1.56788728]]
loss =  [[1.65937546]]
loss =  [[1.75041703]]
loss =  [[1.8437843]]
loss =  [[1.93791809]]
loss =  [[2.02937091]]
loss =  [[2.12125126]]
loss =  [[2.21414371]]
loss =  [[2.30596574]]
loss =  [[2.39775636]]
loss =  [[2.4907405]]
loss =  [[2.58220992]]
loss =  [[2.67458606]]
[[2.76638147]] should be around .26
done 15


# 16

In [14]:
def eval_perfs(y, lbl):
    predict = onehot2label(y)
    print('y.shape = ',y.shape)
    print('predict.shape = ',predict.shape)
    print('lbl.shape = ',lbl.shape)
    perfs = np.sum(predict == lbl)/predict.shape[0]
    perfs = 1-perfs
    return perfs
print(eval_perfs(yinit, ltrain))
print('done 16')

y.shape =  (10, 60000)
predict.shape =  (60000,)
lbl.shape =  (60000,)
0.9353333333333333
done 16


# 17

In [15]:
def update_shallow(x, d, net, gamma=.05):
    W1 = net[0]
    b1 = net[1]
    W2 = net[2]
    b2 = net[3]
    Ni = W1.shape[1]
    Nh = W1.shape[0]
    No = W2.shape[0]
    gamma = gamma / x.shape[1] # normalized by the training dataset size
    # x has shape (784,6000)
    # d has shape (10,6000)
    lbl = label2onehot(d)
    print('x.shape[1] = ', x.shape[1])
    for s in range (x.shape[1]): # update for each sample 
        if s%2000==0:
            print('s = ', s)
        xs = np.reshape(x[:,s],(x[:,s].shape[0],1))
        a1 = W1.dot(xs) + b1
        h1 = relu(a1)
        a2 = W2.dot(h1) + b2
        y = softmax(a2)
        label = np.reshape(lbl[:,s],(lbl[:,s].shape[0],1))
        e = -np.divide(label,y)
        # Backward phase
        delta2 = softmaxp(a2,e) 
        delta1 = relup(a1,h1) * np.transpose(W2).dot(delta2)
        # gradient update
        W2 = W2 - gamma * delta2.dot(h1.T)
        W1 = W1 - gamma * delta1.dot(xs.T)  
        b2 = b2 - gamma * delta2.sum( axis =1 , keepdims = True )
        b1 = b1 - gamma * delta1.sum( axis =1 , keepdims = True )
    return W1, b1, W2, b2
print('done 17')

done 17


# 18

In [ ]:
def backprop_shallow(x, d, net, T, gamma=.05):
    print('backprop_shallow')
    lbl = onehot2label(d)
    y = forwardprop_shallow(x, net)
    for t in range(T):
        print('t = ',t)
        net = update_shallow(x, lbl, net, gamma)
        y = forwardprop_shallow(x, net)
        print('eval_loss(y,d)', eval_loss(y, d))
        print('eval_perfs(y,lbl)', eval_perfs(y, lbl))
    return net
print('ccc')
nettrain = backprop_shallow(xtrain_normalized, dtrain, netinit, 100)
print('done 18')

ccc
backprop_shallow
t =  0
x.shape[1] =  60000
s =  0
s =  2000
s =  4000
s =  6000
s =  8000
s =  10000
s =  12000
s =  14000
s =  16000
s =  18000
s =  20000
s =  22000
s =  24000
s =  26000
s =  28000
s =  30000
s =  32000
s =  34000
s =  36000
s =  38000
s =  40000
s =  42000
s =  44000
s =  46000
s =  48000
s =  50000
s =  52000
s =  54000
s =  56000
s =  58000
y.shape =  (10, 60000)
d.shape =  (10, 60000)
loss =  [[4.91230636e-05]]
loss =  [[0.0773207]]
loss =  [[0.15479363]]
loss =  [[0.23245027]]
loss =  [[0.31059351]]
loss =  [[0.3884185]]
loss =  [[0.46595468]]
loss =  [[0.54371831]]
loss =  [[0.62150855]]
loss =  [[0.69899683]]
loss =  [[0.77719023]]
loss =  [[0.85496509]]
loss =  [[0.93309139]]
loss =  [[1.01100073]]
loss =  [[1.08918733]]
loss =  [[1.16737347]]
loss =  [[1.24501824]]
loss =  [[1.32340502]]
loss =  [[1.40117562]]
loss =  [[1.47901637]]
loss =  [[1.5570276]]
loss =  [[1.63544791]]
loss =  [[1.7130952]]
loss =  [[1.79076864]]
loss =  [[1.8689429]]
loss =  [[

y.shape =  (10, 60000)
predict.shape =  (60000,)
lbl.shape =  (60000,)
eval_perfs(y,lbl) 0.7060500000000001
t =  7
x.shape[1] =  60000
s =  0
s =  2000
s =  4000
s =  6000
s =  8000
s =  10000
s =  12000
s =  14000
s =  16000
s =  18000
s =  20000
s =  22000
s =  24000
s =  26000
s =  28000
s =  30000
s =  32000
s =  34000
s =  36000
s =  38000
s =  40000
s =  42000
s =  44000
s =  46000
s =  48000
s =  50000
s =  52000
s =  54000
s =  56000
s =  58000
y.shape =  (10, 60000)
d.shape =  (10, 60000)
loss =  [[4.50443046e-05]]
loss =  [[0.06983322]]
loss =  [[0.13944556]]
loss =  [[0.20896427]]
loss =  [[0.27902735]]
loss =  [[0.34882127]]
loss =  [[0.41851346]]
loss =  [[0.48872938]]
loss =  [[0.55890056]]
loss =  [[0.62912637]]
loss =  [[0.6988321]]
loss =  [[0.76848547]]
loss =  [[0.83882359]]
loss =  [[0.90902774]]
loss =  [[0.97941943]]
loss =  [[1.04956415]]
loss =  [[1.12030116]]
loss =  [[1.19092245]]
loss =  [[1.26108364]]
loss =  [[1.33135148]]
loss =  [[1.4014548]]
loss =  [[1.

# 19

In [ ]:
xtest, ltest = MNISTtools.load(dataset="testing", path="/datasets/MNIST/")
print(xtest.shape)
print(ltest.shape)
xtest_normalized = normalize_MNIST_images(xtest)
ytest = forwardprop_shallow(xtest_normalized, net)
print(eval_perfs(ytest, ltest))
print('done 19')

1.What is the size of the testing set?
<br>ans : 10k
<br>2.Evaluate the performance of your network on the testing dataset.

# 20

In [ ]:
def backprop_minibatch_shallow(x, d, net, T, B=100, gamma=.05):
    N = x.shape[1]
    NB = int((N+B-1)/B)
    lbl = onehot2label(d)
    for t in range(T):
        shuffled_indices = np.random.permutation(range(N))
        for l in range(NB):
            minibatch_indices = shuffled_indices[B*l:min(B*(l+1), N)]
            x_minibatch = x[:,minibatch_indices]
            lbl_minibatch = lbl[:,minibatch_indices]
            net = update_shallow(x_minibatch, lbl_minibatch, net, gamma)
        y = forwardprop_shallow(x, net)
        print('eval_loss(y,d)', eval_loss(y, d))
        print('eval_perfs(y,lbl)', eval_perfs(y, lbl))
    return net
netminibatch = backprop_minibatch_shallow(xtrain_normalized, dtrain, netinit, 5, B=100)
print('done 20')

# 21

In [ ]:
ytest = forwardprop_shallow(xtest_normalized, net)
print(eval_perfs(ytest, ltest))
print('done 21')